# Predict the year from the abstract 

In [2]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time

import string # библиотека для работы со строками
import nltk   # Natural Language Toolkit
# загружаем библиотеку для лемматизации
import pymorphy2 # Морфологический анализатор

#вычисляем tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Binarizer

In [3]:
from data_types import AuthorsDB
from data_types import PublicationsDB
from data_types import AbstractsDB

audb = AuthorsDB()
audb.load()
pubdb = PublicationsDB()
pubdb.load()
absdb = AbstractsDB()
absdb.load()

filename = "../data/mathnet_iam_authors_dict.pkl"
with open(filename,'rb') as inp:
    authors_dict = pickle.load(inp)

In [4]:
pub = pd.DataFrame.from_dict(pubdb.db,orient='index')

# 1 Data Preprocessing

I need to get 
- abstract and it's tokenization
- year of paper



In [41]:
abstracts = absdb.db[absdb.db['abstract'].notna()].copy()

In [42]:
absdb.db.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7597 entries, vyurv213 to vmumm4420
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   abstract  5609 non-null   object
 1   keywords  2525 non-null   object
dtypes: object(2)
memory usage: 178.1+ KB


In [43]:
abstracts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5609 entries, vyurv213 to da31
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   abstract  5609 non-null   object
 1   keywords  2525 non-null   object
dtypes: object(2)
memory usage: 131.5+ KB


In [6]:
# загружаем список стоп-слов для русского
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')

# примеры стоп-слов
stop_words[:10]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\-\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [7]:
# знаки препинания
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
word_tokenizer = nltk.WordPunctTokenizer()

In [91]:
# dates = [str(x) for x in np.arange(1900, 2022)]
def process_data(data):
    texts = []    
    
    # поочередно проходим по всем новостям в списке
    for item in data:
               
        text_lower = item.lower() # приводим все слова к нижнему регистру
        tokens     = word_tokenizer.tokenize(text_lower) #разбиваем екст на слова
        
        # удаляем пунктуацию и стоп-слова
        # tokens = [word for word in tokens if (word not in string.punctuation and word not in stop_words and word not in dates)]
        tokens = [word for word in tokens if (word not in string.punctuation and word not in stop_words)]
        
        texts.append(tokens) # добавляем в предобработанный список
    
    return texts

def process_abstractsdb(data):
    texts = []    
    
    # поочередно проходим по всем новостям в списке
    for index,row in data.iterrows():
        # print(row['abstract'])        
        print(type(row['abstract']))
        text = row['abstract']
        if (row['keywords'] is not None):
            # print(text)
            # print(row['keywords'])
            print(type(row['keywords']))
            text += row['keywords']                   
        text_lower = text.lower() # приводим все слова к нижнему регистру
        tokens     = word_tokenizer.tokenize(text_lower) #разбиваем екст на слова                
        tokens = [word for word in tokens if (word not in string.punctuation and word not in stop_words and not word.isnumeric())]
                      
        texts.append(tokens) # добавляем в предобработанный список
    
    return texts

In [92]:
abstracts.loc[['vyurv213','vyurv1','vyurv46']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, vyurv213 to vyurv46
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   abstract  3 non-null      object
 1   keywords  3 non-null      object
dtypes: object(2)
memory usage: 72.0+ bytes


In [93]:
exp = ["Теперь каждый пример представлен списком слов.",
       "Причем все слова с маленькой буквы.",
       "Пунктуацию и стоп-слова мы удалили."]
texts_exp = process_data(exp)

# texts = process_abstractsdb(abstracts.loc[['vyurv213','vyurv1','vyurv46']])
texts = process_abstractsdb(abstracts.loc[['vyurv213','vyurv1','vyurv46']])

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [94]:
print(texts_exp)

[['каждый', 'пример', 'представлен', 'списком', 'слов'], ['причем', 'слова', 'маленькой', 'буквы'], ['пунктуацию', 'стоп', 'слова', 'удалили']]


In [95]:
print(texts)
texts_ll = texts[:]

[['увеличивающийся', 'рост', 'числа', 'компонент', 'суперкомпьютеров', 'приводит', 'специалистов', 'области', 'hpc', 'неблагоприятным', 'оценкам', 'будущих', 'суперкомпьютеров', 'диапазон', 'среднего', 'времени', 'отказами', 'составлять', 'часа', 'часов', 'данная', 'оценка', 'ставит', 'вопрос', 'возможность', 'проведения', 'длительных', 'расчетов', 'суперкомпьютерах', 'работе', 'предлагается', 'метод', 'восстановления', 'отказов', 'требующий', 'возврата', 'большинства', 'процессов', 'последней', 'контрольной', 'точке', 'позволить', 'сократить', 'накладные', 'расходы', 'некоторых', 'вычислительных', 'алгоритмов', 'стандартный', 'метод', 'обеспечения', 'отказоустойчивости', 'заключается', 'координированном', 'сохранении', 'случае', 'отказа', 'осуществляется', 'возврат', 'процессов', 'последней', 'контрольной', 'точке', 'предлагаемая', 'стратегия', 'заключается', 'координированном', 'сохранении', 'журналировании', 'передаваемых', 'данных', 'случае', 'отказа', 'происходит', 'асинхронное', 

In [96]:
# инициализируем лемматизатор :)
morph = pymorphy2.MorphAnalyzer()

In [99]:
for aword in texts[2]:
    aword_norm = morph.parse(aword)[0].normal_form
    print("Исходное слово: %s \tЛемматизированное: %s" % (aword, aword_norm))

Исходное слово: рассматриваются 	Лемматизированное: рассматриваться
Исходное слово: вопросы 	Лемматизированное: вопрос
Исходное слово: связанные 	Лемматизированное: связанный
Исходное слово: проведением 	Лемматизированное: проведение
Исходное слово: расчетов 	Лемматизированное: расчёт
Исходное слово: распределенных 	Лемматизированное: распределённый
Исходное слово: вычислительных 	Лемматизированное: вычислительный
Исходное слово: системах 	Лемматизированное: система
Исходное слово: компоненты 	Лемматизированное: компонент
Исходное слово: которых 	Лемматизированное: который
Исходное слово: подвержены 	Лемматизированное: подверженный
Исходное слово: отказам 	Лемматизированное: отказ
Исходное слово: работе 	Лемматизированное: работа
Исходное слово: приводятся 	Лемматизированное: приводиться
Исходное слово: определения 	Лемматизированное: определение
Исходное слово: системы 	Лемматизированное: система
Исходное слово: сбоя 	Лемматизированное: сбой
Исходное слово: ошибки 	Лемматизированное: 

In [100]:
# применяем лемматизацию ко всем текстам
print(range(len(texts)))

for i in tqdm(range(len(texts))):           # tqdm_notebook создает шкалу прогресса :)
    text_lemmatized = [morph.parse(x)[0].normal_form for x in texts[i]] # применяем лемматизацию для каждого слова в тексте
    texts[i] = ' '.join(text_lemmatized)
    # time.sleep(1)  

range(0, 3)


  0%|          | 0/3 [00:00<?, ?it/s]

In [101]:
print(texts)

['увеличиваться рост число компонент суперкомпьютер приводить специалист область hpc неблагоприятный оценка будущий суперкомпьютер диапазон среднее время отказ составлять час час дать оценка ставить вопрос возможность проведение длительный расчёт суперкомпьютер работа предлагаться метод восстановление отказ требовать возврат большинство процесс последний контрольный точка позволить сократить накладный расход некоторый вычислительный алгоритм стандартный метод обеспечение отказоустойчивость заключаться координировать сохранение случай отказ осуществляться возврат процесс последний контрольный точка предлагать стратегия заключаться координировать сохранение журналирование передаваемый данные случай отказ происходить асинхронный восстановление асинхронный восстановление несколько запасный процесс проводить пересчёт данные потерянный отказ остальной процесс находиться ожидание окончание процедура восстановление потерянный данные разработать параллельный программа решающий задача распростра

https://stackoverflow.com/questions/46118910/scikit-learn-vectorizer-max-features

In [107]:
# Fit TF-IDF on train texts
# vectorizer = TfidfVectorizer(min_df=1,   max_features=None, strip_accents='unicode',  analyzer='word',token_pattern=r'\w{2,}',ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1)
vectorizer = TfidfVectorizer(min_df=1,   max_features=None, analyzer='word',token_pattern=r'\w{2,}', use_idf=1,smooth_idf=1,sublinear_tf=1) 
vectorizer.fit(texts)


TfidfVectorizer(smooth_idf=1, sublinear_tf=1, token_pattern='\\w{2,}',
                use_idf=1)

In [108]:
print(len(vectorizer.get_feature_names_out()))


190


In [109]:

# Топ-10 слов
vectorizer.get_feature_names_out()[:]

array(['hpc', 'mpi', 'raise', 'sigkill', 'ulfm', 'автоматический',
       'алгоритм', 'асинхронный', 'библиотека', 'больший', 'большинство',
       'будущий', 'важный', 'возврат', 'возможность', 'вопрос',
       'восстановить', 'восстановление', 'время', 'вызов', 'выполнение',
       'высокопроизводительный', 'вычисление', 'вычислительный',
       'глобальный', 'группа', 'данные', 'данный', 'дать', 'диапазон',
       'диск', 'длительный', 'допустимый', 'дублирование', 'её',
       'журналирование', 'задача', 'заключаться', 'запасный', 'запись',
       'запускать', 'значение', 'имитационный', 'исполнение',
       'использовать', 'исследование', 'компонент', 'контрольный',
       'координировать', 'который', 'кратко', 'локальный', 'метод',
       'моделирование', 'моделирующий', 'модель', 'модификация',
       'наиболее', 'накладный', 'находиться', 'неблагоприятный',
       'незначительный', 'некоторый', 'непосредственный', 'несколько',
       'обеспечение', 'область', 'обработка', 'объё

In [110]:
train_X = vectorizer.fit_transform(texts)

In [111]:
print(type(train_X))

<class 'scipy.sparse._csr.csr_matrix'>


In [112]:
train_X.shape

(3, 190)

In [203]:
def save_vectorizer(data,status):
    filename = '../data/vectorizer.pkl'    
    if status == 'write':        
        with open(filename,'wb') as outp:
            pickle.dump(data, outp, pickle.HIGHEST_PROTOCOL)

In [204]:
save_vectorizer(vectorizer,status="write")

In [205]:
def load_vectorizer():
    filename = '../data/vectorizer.pkl'
    res = None    
    with open(filename,'rb') as inp:
        res = pickle.load(inp)
    return res

In [209]:
# x = load_vectorizer()

In [210]:
# x.get_feature_names_out()[:10]

array(['hpc', 'mpi', 'raise', 'sigkill', 'ulfm', 'автоматический',
       'алгоритм', 'асинхронный', 'библиотека', 'больший'], dtype=object)

# Importance of the article for Request

In [211]:

# from sklearn.feature_extraction.text import CountVectorizer

# freq   = CountVectorizer()
# corpus = freq.fit_transform(corpus)
# X =['причём стоп слово удалить буква'] 
request =['суперкомпьютерное моделирование отказоустойчивых вычислений на hpc системах'] 
train_y = vectorizer.transform(X)

onehot = Binarizer()
ohe_request = onehot.fit_transform(train_y.toarray())


In [212]:
# print(corpus)
# print(type(corpus))

In [213]:
print(ohe_request.shape)
print(train_X.shape)

(1, 190)
(3, 190)


In [214]:
answer= train_X.dot(ohe_request.T)
print(answer)

[[0.        ]
 [0.21686795]
 [0.        ]]


In [160]:
answer = answer.flatten()

In [177]:
pubs_index = np.argsort(answer)[::-1][:5]
print(pubs_index)

[1 2 0]


In [178]:
req_pups = list(absdb.db.iloc[pubs_index].index)
print(req_pups)

['vyurv1', 'vyurv46', 'vyurv213']


In [179]:
answer[pubs_index]

array([0.21686795, 0.        , 0.        ])

In [175]:
# pub.info()
pub.loc[req_pups];

In [200]:
from collections import defaultdict
def get_authors_rating(df, coef):
    # print(df)
    # print(type(df))
    # print(coef)
    # print(type(coef))
    rating = defaultdict(int)
    for index in range(len(df)):
        # print(f'index = {index}, row = {df.iloc[index]}')        
        for author in df.iloc[index]['author_id']:  
            # print(index)          
            print(f'add coef = {coef[index]} to author = {author}')
            rating[author] += coef[index]            
            print(rating)
    return rating

In [202]:
res = get_authors_rating(pub.loc[req_pups],answer[pubs_index])
print(res)

add coef = 0.21686794707455037 to author = 113970
defaultdict(<class 'int'>, {'113970': 0.21686794707455037})
add coef = 0.21686794707455037 to author = 22428
defaultdict(<class 'int'>, {'113970': 0.21686794707455037, '22428': 0.21686794707455037})
add coef = 0.0 to author = 113970
defaultdict(<class 'int'>, {'113970': 0.21686794707455037, '22428': 0.21686794707455037})
add coef = 0.0 to author = 22428
defaultdict(<class 'int'>, {'113970': 0.21686794707455037, '22428': 0.21686794707455037})
add coef = 0.0 to author = 113970
defaultdict(<class 'int'>, {'113970': 0.21686794707455037, '22428': 0.21686794707455037})
add coef = 0.0 to author = 148811
defaultdict(<class 'int'>, {'113970': 0.21686794707455037, '22428': 0.21686794707455037, '148811': 0.0})
add coef = 0.0 to author = 22428
defaultdict(<class 'int'>, {'113970': 0.21686794707455037, '22428': 0.21686794707455037, '148811': 0.0})
defaultdict(<class 'int'>, {'113970': 0.21686794707455037, '22428': 0.21686794707455037, '148811': 0.0}

In [129]:
# train_X.todense()[2,[20, 19, 18, 15, 13,  9,  0]].sum()
# train_X.todense()[2,[20, 19, 18, 15, 13,  9,  0]]